In [86]:
# render_minicharts.py
# Requisitos: matplotlib, numpy, pillow
# Uso: python render_minicharts.py  (gera serasa_finscore_side_by_side.png)

import matplotlib
matplotlib.use("Agg")  # <- backend não-interativo, funciona em qualquer ambiente

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from PIL import Image

# Paletas pastel
TEAL_PASTEL = ["#BFEDE6", "#A6E4DB", "#8ADBCF", "#6FD2C4"]          # Serasa (verde-água)
BLUE_PASTEL = ["#C8DBF4", "#AECBEE", "#90BAE8", "#78A6DB", "#5F90CE"]  # FinScore (azuis)

def draw_minichart(ax, categories, values, colors, score,
                   font_x=7, font_value=8, grid_color="#EDEFF3",
                   score_color="#7A8596"):
    """
    Mini bar chart:
      - barras partem do eixo x (sem bordas)
      - linha pontilhada no 'score'
      - valores pequenos acima das barras
      - sem spines e sem ticks no eixo y
    """
    x = np.arange(len(categories))
    ymax = max(max(values), float(score)) * 1.15

    # Barras (sem borda)
    ax.bar(x, values, width=0.55, color=colors[:len(values)],
           edgecolor="none", linewidth=0.0, zorder=3)

    # Grid leve
    ax.grid(axis="y", color=grid_color, linestyle="-", linewidth=1, zorder=0)

    # Linha do resultado (pontilhada)
    ax.axhline(score, linestyle="--", color=score_color, linewidth=1.2, zorder=5)

    # Rótulos de valor
    for i, v in enumerate(values):
        ax.text(i, v + ymax*0.03, f"{v:.0f}", ha="center", va="bottom",
                fontsize=font_value, color="#8A8F99", fontweight="bold", zorder=5)

    # Eixo x
    ax.set_xticks(x)
    ax.set_xticklabels(categories, fontsize=font_x, color="#1E2A3A")
    ax.set_ylim(0, ymax)
    ax.set_yticks([])

    # Sem bordas
    for spine in ax.spines.values():
        spine.set_visible(False)

    ax.set_facecolor("#FDFAFB")


def make_serasa_finscore_png(
    serasa_score=550,
    finscore_score=905,
    serasa_vals=(300, 500, 700, 1000),
    finscore_vals=(125, 250, 750, 875, 1000),
    out_path="serasa_finscore_side_by_side.png"
):
    # Categorias
    serasa_cats = ["Muito Baixo", "Baixo", "Bom", "Excelente"]
    finscore_cats = ["M. Acima", "L. Acima", "Neutro", "L. Abaixo", "M. Abaixo"]

    # Dois eixos lado a lado, bem próximos
    fig, (axL, axR) = plt.subplots(
        1, 2, figsize=(10.0, 3.0), dpi=250, gridspec_kw=dict(wspace=0.06)
    )
    fig.patch.set_facecolor("#FDFAFB")  # mesmo fundo dos gráficos

    # Desenho dos minicharts
    draw_minichart(axL, serasa_cats, serasa_vals, TEAL_PASTEL, score=serasa_score)
    draw_minichart(axR, finscore_cats, finscore_vals, BLUE_PASTEL, score=finscore_score)

    # Margens (evite tight_layout; controlamos manualmente)
    plt.subplots_adjust(left=0.06, right=0.99, top=0.92, bottom=0.30, wspace=0.06)

    # Logos embaixo dos gráficos (centralizados e pequenos)
    try:
        # Logo Serasa - carregar direto como RGB (já foi convertido)
        logo_serasa = Image.open(r"C:\Users\ferna\OneDrive\Desktop OneDrive\FinScore\app_front\assets\logo_serasa.png")
        logo_serasa = logo_serasa.convert("RGB")  # Forçar RGB
        imagebox_serasa = OffsetImage(logo_serasa, zoom=0.06)
        boxL = axL.get_position()
        # Posição em coordenadas da figura (60px para a esquerda, 20px para baixo)
        offset_x = 60 / (fig.get_figwidth() * fig.dpi)
        offset_y = 20 / (fig.get_figheight() * fig.dpi)
        ab_serasa = AnnotationBbox(imagebox_serasa, 
                                    xy=(boxL.x0 + boxL.width/2 - offset_x, boxL.y0 - 0.10 - offset_y),
                                    xycoords='figure fraction',
                                    frameon=False,
                                    box_alignment=(0.5, 1.0))
        fig.add_artist(ab_serasa)
        
        # Logo FinScore - adicionar fundo com a cor do gráfico (#FDFAFB)
        logo_finscore = Image.open(r"C:\Users\ferna\OneDrive\Desktop OneDrive\FinScore\app_front\assets\logo_fin1a.png")
        if logo_finscore.mode != 'RGBA':
            logo_finscore = logo_finscore.convert("RGBA")
        bg_finscore = Image.new("RGB", logo_finscore.size, (253, 250, 251))
        bg_finscore.paste(logo_finscore, (0, 0), logo_finscore)
        logo_finscore = bg_finscore
        imagebox_finscore = OffsetImage(logo_finscore, zoom=0.06)
        boxR = axR.get_position()
        ab_finscore = AnnotationBbox(imagebox_finscore,
                                      xy=(boxR.x0 + boxR.width/2 - offset_x, boxR.y0 - 0.10 - offset_y),
                                      xycoords='figure fraction',
                                      frameon=False,
                                      box_alignment=(0.5, 1.0))
        fig.add_artist(ab_finscore)
    except Exception as e:
        print(f"Aviso: Não foi possível carregar os logos: {e}")
        # Fallback para texto caso as imagens não sejam encontradas
        fig.canvas.draw()
        boxL = axL.get_position()
        boxR = axR.get_position()
        fig.text(boxL.x0 + boxL.width/2, boxL.y0 - 0.10, "Serasa",
                 ha="center", va="top", fontsize=10, fontweight="bold", color="#303030")
        fig.text(boxR.x0 + boxR.width/2, boxR.y0 - 0.10, "FinScore",
                 ha="center", va="top", fontsize=10, fontweight="bold", color="#303030")

    # Salva imagem e encerra (facecolor garante fundo sem transparência)
    fig.savefig(out_path, bbox_inches="tight", facecolor=fig.get_facecolor(), edgecolor='none')
    plt.close(fig)
    return out_path


if __name__ == "__main__":
    path = make_serasa_finscore_png(
        serasa_score=550,
        finscore_score=905,
        serasa_vals=(300, 500, 700, 1000),
        finscore_vals=(125, 250, 750, 875, 1000),
        out_path="serasa_finscore_side_by_side.png"
    )
    print(f"OK! Arquivo gerado em: {path}")


OK! Arquivo gerado em: serasa_finscore_side_by_side.png
